In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sales = pd.read_csv('../data/read_summarized_sales_clean.csv')
cci = pd.read_csv('../data/read_cci_clean.csv')
gdp = pd.read_csv('../data/read_gdp_sector_clean.csv')
rates = pd.read_csv('../data/read_fed_funds_rate_clean.csv')
jobs = pd.read_csv('../data/read_nonfarm_jobs_clean.csv')
fuel = pd.read_csv('../data/read_jet_fuel_clean.csv')
unemployment = pd.read_csv('../data/read_unemployment_rate_clean.csv')

**Revenue**

In [6]:
sales.head()

,Year,Month,Qtr,Year-Month,Year-Qtr,Customer,S_Cons_Order_Class,Market,Total Rev,Order Count
0,2021,Jan,Q1,2021-Jan,2021-Q1,"al-Ahsan, Rashaad",At-Once,Non-Profit,5739.30,1
1,2021,Jan,Q1,2021-Jan,2021-Q1,"al-Fahmy, Haazim",At-Once,Prof Serv,660.00,1
2,2021,Jan,Q1,2021-Jan,2021-Q1,"al-Hamed, Zeena",At-Once,Local,0.00,1
3,2021,Jan,Q1,2021-Jan,2021-Q1,"al-Hamed, Zeena",In-Stock,Food/Bev,793.45,2
4,2021,Jan,Q1,2021-Jan,2021-Q1,"al-Hamed, Zeena",In-Stock,General,286.25,1


In [8]:
month_rev = sales.groupby('Year-Month').agg({
    'Total Rev': 'sum',
    'Order Count': 'sum'
}).reset_index()
qtr_rev = sales.groupby('Year-Qtr').agg({
    'Total Rev': 'sum',
    'Order Count': 'sum'
}).reset_index()

In [10]:
month_rev.head()

,Year-Month,Total Rev,Order Count
0,2021-Apr,3266201.33,10326
1,2021-Aug,3580439.44,13230
2,2021-Dec,3980600.52,10168
3,2021-Feb,2062865.99,6491
4,2021-Jan,2329337.30,7064


In [12]:
qtr_rev.head()

,Year-Qtr,Total Rev,Order Count
0,2021-Q1,6797150.27,20839
1,2021-Q2,9221239.39,44814
2,2021-Q3,10458666.22,44348
3,2021-Q4,11123543.36,32468
4,2022-Q1,10487291.42,24253


**Consumer Confidence Index**

In [15]:
cci.head()

,Category,Year,Month,Year-Month,CCI
0,1/1/2021,2021,Jan,2021-Jan,99.25391
1,2/1/2021,2021,Feb,2021-Feb,99.39012
2,3/1/2021,2021,Mar,2021-Mar,99.74757
3,4/1/2021,2021,Apr,2021-Apr,99.98598
4,5/1/2021,2021,May,2021-May,99.92869


In [ ]:
cci_rev = pd.merge(

In [ ]:
gdp.head()

In [ ]:
rates.head()

In [ ]:
jobs.head()

In [ ]:
fuel.head()

In [ ]:
unemployment.head()